<a href="https://colab.research.google.com/github/Tclack88/DS-Unit-4-Sprint-1-NLP/blob/master/module2-vector-representations/LS_DS_412_Vector_Representations_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>

# Vector Representations
## *Data Science Unit 4 Sprint 2 Assignment 2*

In [1]:
# !python -m spacy download en_core_web_lg

     |████████████████████████████████| 826.9MB 13.2MB/s 
  Created wheel for en-core-web-lg: filename=en_core_web_lg-2.1.0-cp36-none-any.whl size=828255076 sha256=8232407cb5efc598e38fd07c7ab7ed9993df89ff2ddaf0e455e8a89910166843
  Stored in directory: /tmp/pip-ephem-wheel-cache-07mrg99l/wheels/b4/d7/70/426d313a459f82ed5e06cc36a50e2bb2f0ec5cb31d8e0bdf09
Successfully built en-core-web-lg
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [0]:
import re
import string

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import spacy

## 1) *Optional:* Scrape 100 Job Listings that contain the title "Data Scientist" from indeed.com

At a minimum your final dataframe of job listings should contain
- Job Title
- Job Description

If you choose to not to scrape the data, there is a CSV with outdated data in the directory. Remeber, if you scrape Indeed, you're helping yourself find a job. ;)

In [3]:
!pip install requests_html

     |████████████████████████████████| 1.2MB 13.7MB/s 
     |████████████████████████████████| 81kB 8.2MB/s 
  Created wheel for parse: filename=parse-1.12.1-cp36-none-any.whl size=22342 sha256=73eaa51f8fa412b6747ab20626dda7c097a838cd24605c3090d5046027eaed82
  Stored in directory: /root/.cache/pip/wheels/16/f1/d5/72f7e2c22ab3d106bb6f7d727552b5e027b832284f4704b106
  Created wheel for fake-useragent: filename=fake_useragent-0.1.11-cp36-none-any.whl size=13485 sha256=1a9d63a0dcf5a0f440d6a0e704dbf4f97cc16c65b55245149b71b4d918df316e
  Stored in directory: /root/.cache/pip/wheels/5e/63/09/d1dc15179f175357d3f5c00cbffbac37f9e8690d80545143ff
  Created wheel for pyppeteer: filename=pyppeteer-0.0.25-cp36-none-any.whl size=78362 sha256=b3ab0654d7cf72a9a96fcd55b5978ada0effa1c2a6b666407c1258885d2ba575
  Stored in directory: /root/.cache/pip/wheels/34/e0/5d/070e22eceecf7ecd5fa4b86bbc18c1c7d0b90e02e9b57f35eb
Successfully built parse fake-useragent pyppeteer


## Intellectual property below "borrowed" and modified from Richmond Macaspac

In [0]:
from urllib.parse import urlencode
from requests_html import HTMLSession
from multiprocessing.dummy import Pool
from itertools import chain
class IndeedJobListings:
    '''
    Intellectual property of Richmond Macaspac
    Multi-theaded Indeed Job Listings Crawler
    Usage:
    listings = IndeedJobListings('Data Scientist', 'Irvine, CA')
    descriptions = listings.get_descriptions(pages=10)
    '''
    def __init__(self, search_keyword, location, threads=12):
        self.threads = threads
        self.base_url = 'https://www.indeed.com'
        self.query_url = f'{self.base_url}/jobs?' +\
        urlencode({'q': search_keyword, 'l': location})
        self.session = HTMLSession()
    def _get_posting_urls(self, url):
        doc = self.session.get(url)
        posting_urls = [f'{self.base_url}{e.attrs["href"]}' for e in doc.html.find('.jobtitle.turnstileLink')]
        return posting_urls
    def _get_description_text(self, url):
        doc = self.session.get(url)
        description_text = doc.html.find('#jobDescriptionText')[0].text
        return description_text
    def _get_job_title(self, url):
        doc = self.session.get(url)
        job_title = doc.html.find('title')[0].text.split(' - ')[0]
        return job_title
    def get_job_titles(self, pages=1):
        list_urls = [self.query_url] + [f'{self.query_url}&start={x*10}'
                                        for x in range(1, pages)]
        p = Pool(self.threads)
        post_urls = chain(*p.map(self._get_posting_urls, list_urls))
        job_title = p.map(self._get_job_title, post_urls)
        return job_title
    def get_descriptions(self, pages=1):
        list_urls = [self.query_url] + [f'{self.query_url}&start={x*10}'
                                        for x in range(1, pages)]
        p = Pool(self.threads)
        post_urls = chain(*p.map(self._get_posting_urls, list_urls))
        descriptions = p.map(self._get_description_text, post_urls)
        return descriptions

listings = IndeedJobListings('Data Scientist', 'Irvine, CA')

descriptions = listings.get_descriptions(pages=8)
job_titles = listings.get_job_titles(pages=8)

In [16]:
ds_jobs = pd.DataFrame(list(zip(job_titles, descriptions)), columns=['title','job_desc'])
ds_jobs = ds_jobs.head(100)
ds_jobs.head()

,title,job_desc
0,Data Scientist II,"Teamwork. Integrity. Dedication.\nTogether, we..."
1,Senior Data Analyst,Qualifications & Responsibilities\n\n\n\nThe A...
2,Senior HR Data Analyst,Data Scientist:\n\nAlignment Healthcare is a d...
3,Senior Data Scientist,"Data Scientist | Acorns\n\nAt Acorns, we're bu..."
4,Data Scientist,Under the direction of the Director of Enrollm...


## 2) Use Spacy to tokenize / clean the listings 

In [0]:
nlp = spacy.load('en_core_web_lg')
tokenizer = spacy.tokenizer.Tokenizer(nlp.vocab)
# def cleanup(df):
#   df.job_desc = df.job_desc.apply(lambda x: re.sub('

In [18]:
STOPWORDS = nlp.Defaults.stop_words.union(['data','science','scientist'])


def cleanup(df):
  df.job_desc = df.job_desc.apply(lambda x: re.sub(r'\n',' ',x).strip().lower())
  df.job_desc = df.job_desc.apply(lambda x: re.sub('[^a-z 0-9]','',x))
  tokens = []
  for doc in tokenizer.pipe(df.job_desc):
    doc_tokens = [token.lemma_ for token in doc if (token.text not in STOPWORDS) and (token.is_punct == False)]
    tokens.append(doc_tokens)
  df['tokens'] = tokens
  return df

ds_jobs = cleanup(ds_jobs)

ds_jobs.head()

,title,job_desc,tokens
0,Data Scientist II,teamwork integrity dedication together we make...,"[teamwork, integrity, dedication, difference, ..."
1,Senior Data Analyst,qualifications responsibilities the analyt...,"[qualification, , responsibility, , analyt..."
2,Senior HR Data Analyst,data scientist alignment healthcare is a data...,"[ , alignment, healthcare, technology, drive, ..."
3,Senior Data Scientist,data scientist acorns at acorns were buildin...,"[ , acorn, , acorn, build, financial, wellnes..."
4,Data Scientist,under the direction of the director of enrollm...,"[direction, director, enrollment, management, ..."


## 3) Use Scikit-Learn's CountVectorizer to get word counts for each listing.

In [0]:
# In order for the count vectorizer to work, we need to convert our list of tokens into a string
def tokens_to_string(tokens):
  return ' '.join(tokens) 

ds_jobs.tokens = ds_jobs.tokens.apply(tokens_to_string)

In [22]:
ds_jobs

,title,job_desc,tokens
0,Data Scientist II,teamwork integrity dedication together we make...,teamwork integrity dedication difference caree...
1,Senior Data Analyst,qualifications responsibilities the analyt...,qualification responsibility analytic co...
2,Senior HR Data Analyst,data scientist alignment healthcare is a data...,alignment healthcare technology drive health...
3,Senior Data Scientist,data scientist acorns at acorns were buildin...,acorn acorn build financial wellness syste...
4,Data Scientist,under the direction of the director of enrollm...,direction director enrollment management analy...
...,...,...,...
95,Sr. Artificial Intelligence Speech Engineer,data scientist alignment healthcare is a data...,alignment healthcare technology drive health...
96,"Cloud Solutions Architect, Data Intelligence",job title senior hr data analyst job id 101414...,job title senior hr analyst job id 1014146 loc...
97,Data Science Intern,teamwork integrity dedication together we make...,teamwork integrity dedication difference caree...
98,Data Scientist,teamwork integrity dedication together we make...,teamwork integrity dedication difference caree...


In [23]:
from sklearn.feature_extraction.text import CountVectorizer

vect = CountVectorizer()
dtm = vect.fit_transform(ds_jobs.tokens)
df = pd.DataFrame(dtm.todense(), columns = vect.get_feature_names(), index = ds_jobs.title)
df

,10,100,1014146,1099,11,11566,1173,12,14,14000,15,1525,16,170,17th,1889,19029953,1966,1972,1983,1993,1b,20,200,2000,2017,2018,2019,2020,20m,21,223061,23,25,2500,30,3050,31,34,34000plus,...,wide,will,willingness,win,window,wine,woman,womens,word,work,workforce,working,worklife,workload,workplace,world,worldclass,worldrenowned,worldwide,wow,wrangle,write,wwwcareersfirstamcom,wwwchipotlecom,wwwcoxautoinccom,wwwdatatreecom,wwwdhsgoveverify,wwwepsiloncom,wwwsccwrporg,xo,xtime,yamaha,year,youll,youre,youth,youve,zealous,zero,zillow
title,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Data Scientist II,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Senior Data Analyst,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,1,1,0,1,0,0,2,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0
Senior HR Data Analyst,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
Senior Data Scientist,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,8,0,0,0,0,0,3,0,1,0,0,0,2,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
Data Scientist,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Sr. Artificial Intelligence Speech Engineer,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
"Cloud Solutions Architect, Data Intelligence",0,0,1,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,7,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0
Data Science Intern,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0


## 4) Visualize the most common word counts

In [24]:
list(df.columns)
# df.vans.sum()
words = []
for col in list(df.columns):
  words.append((col,df[col].sum()))# for col in list(df.columns))
words = dict(words)
words = sorted(words.items(), key=lambda x: x[1], reverse=True)
words
# df.columns.sum

[('experience', 574),
 ('work', 387),
 ('business', 324),
 ('team', 296),
 ('model', 260),
 ('include', 247),
 ('learn', 247),
 ('skill', 237),
 ('project', 235),
 ('provide', 222),
 ('analysis', 207),
 ('build', 201),
 ('service', 190),
 ('ability', 171),
 ('opportunity', 171),
 ('understand', 169),
 ('market', 166),
 ('analytics', 159),
 ('employee', 154),
 ('solution', 154),
 ('customer', 151),
 ('technology', 151),
 ('member', 150),
 ('machine', 147),
 ('lead', 145),
 ('year', 144),
 ('analytic', 140),
 ('product', 140),
 ('report', 137),
 ('statistical', 132),
 ('big', 127),
 ('company', 127),
 ('development', 123),
 ('drive', 123),
 ('help', 121),
 ('new', 119),
 ('information', 118),
 ('require', 117),
 ('process', 116),
 ('support', 116),
 ('plan', 115),
 ('organization', 114),
 ('strong', 114),
 ('need', 111),
 ('problem', 110),
 ('management', 108),
 ('analyst', 106),
 ('system', 103),
 ('engineer', 102),
 ('state', 101),
 ('computer', 100),
 ('tool', 100),
 ('advance', 99),


## 5) Use Scikit-Learn's tfidfVectorizer to get a TF-IDF feature matrix

In [30]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words='english')
dtm = tfidf.fit_transform(ds_jobs.job_desc)
dtm = pd.DataFrame(dtm.todense(), columns=tfidf.get_feature_names(), index=ds_jobs.title)
dtm.head(5)

,10,100,1014146,1099,11,11566,1173,12,14,14000,15,1525,16,170,17th,1889,19029953,1966,1972,1983,1993,1b,20,200,2000,2017,2018,2019,2020,20m,21,223061,23,25,2500,30,3050,31,34,34000plus,...,womens,word,words,work,workforce,working,workings,worklife,workload,workplaces,works,world,worldclass,worldrenowned,worlds,worldwide,wowing,wrangling,write,writing,written,wwwcareersfirstamcom,wwwchipotlecom,wwwcoxautoinccom,wwwdatatreecom,wwwdhsgoveverify,wwwepsiloncom,wwwsccwrporg,xo,xtime,yamaha,year,years,youll,youre,youth,youve,zealous,zeros,zillow
title,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Data Scientist II,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.052826,0.0,0.0,0.052826,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.04854,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.045312,0.0,0.091433,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Senior Data Analyst,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.017374,0.0,0.023372,0.0,0.0,0.0,0.0,0.0,0.035006,0.050239,0.000000,0.000000,0.046657,0.0,0.0,0.0,0.046657,0.030132,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.040394,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Senior HR Data Analyst,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.050470,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.033897,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.019557,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Senior Data Scientist,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.026906,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.061550,0.0,0.011828,0.0,0.0,0.0,0.0,0.0,0.035433,0.000000,0.034349,0.026132,0.000000,0.0,0.0,0.0,0.023613,0.015250,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.010222,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Data Scientist,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.056130,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## 6) Create a NearestNeighbor Model. Write the description of your ideal datascience job and query your job listings. 

In [31]:
from sklearn.neighbors import NearestNeighbors

nn = NearestNeighbors(n_neighbors=5, algorithm='kd_tree')
nn.fit(dtm)

NearestNeighbors(algorithm='kd_tree', leaf_size=30, metric='minkowski',
                 metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                 radius=1.0)

In [39]:
nn.kneighbors([dtm.iloc[0]])

(array([[0., 0., 0., 0., 0.]]), array([[ 0, 60, 33, 37, 54]]))

In [51]:
ideal_job = ["I want to work for a company that does good for the environment. In my opinion, this can be achieved through application of Machine Learning. While there are many environmental projects which may work, the most immediate application coming to mind is self driving cars. I am also strongly intrested in improving traffic congestion, mainly by getting rid of stoplights altogether and conceiving of a way autonomous vehicles can cross through an intersection without stopping."]

ideal_job = tfidf.transform(ideal_job)
ideal_job
job_matches = list(nn.kneighbors(ideal_job.todense())[1][0])
job_matches

[49, 82, 18, 99, 52]

In [52]:
for job in job_matches:
  print(ds_jobs.iloc[job][:200])

title                                     Senior Data Analyst
job_desc    company summary  join our team first americans...
tokens      company summary   join team americans tree llc...
Name: 49, dtype: object
title                                   Senior Data Scientist
job_desc    company summary  join our team first americans...
tokens      company summary   join team americans tree llc...
Name: 82, dtype: object
title                                        Sr. Data Analyst
job_desc    company summary  join our team first americans...
tokens      company summary   join team americans tree llc...
Name: 18, dtype: object
title                                   Senior Data Scientist
job_desc    company summary  join our team first americans...
tokens      company summary   join team americans tree llc...
Name: 99, dtype: object
title                                          Data Scientist
job_desc    company summary  join our team first americans...
tokens      company summary   join t

In [55]:
# It appears these are all the same job, I will look at the full description however

ds_jobs.iloc[49].job_desc

'company summary  join our team first americans data tree llc is the nations leading provider of property information and online recorded public land records we offer billions of imaged documents along with innovative comprehensive postclosing and servicing solutions including document retrieval data delivery lien release and assignment preparation and document management services as a global leader in providing title insurance settlement services and risk solutions for real estate transactions first american nyse faf is an ideal place to build your career we have been entrusted with helping our customers achieve and protect their dream of homeownership since 1889 we believe that our people are the key to the companys continued success and we invest in diverse talents and backgrounds and empower our teams to achieve more than they could anywhere else first american has created an awardwinning culture and has been named to the fortune 100 best companies to work for list for the fourth c

## Stretch Goals

 - Try different visualizations for words and frequencies - what story do you want to tell with the data?
 - Scrape Job Listings for the job title "Data Analyst". How do these differ from Data Scientist Job Listings
 - Try and identify requirements for experience specific technologies that are asked for in the job listings. How are those distributed among the job listings?
 - Use a clustering algorithm to cluster documents by their most important terms. Do the clusters reveal any common themes?
  - **Hint:** K-means might not be the best algorithm for this. Do a little bit of research to see what might be good for this. Also, remember that algorithms that depend on Euclidean distance break down with high dimensional data.
 - Create a labeled dataset - which jobs will you apply for? Train a model to select the jobs you are most likely to apply for. :) 